In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import special
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import copy

In [2]:
# Existing functions as provided
def generate_digital_signal(length):
    return np.random.choice([-1, 1], size=length)


In [5]:
def Jakes_filter(f_max, Ts, N):
    L = N // 2
    n = np.arange(1, L + 1)
    J_pos = special.jv(0.25, 2 * np.pi * f_max * n * Ts) / (n ** 0.25)
    J_neg = np.flip(J_pos)
    J_0 = 1.468813 * (f_max * Ts) ** 0.25
    J = np.concatenate((J_neg, [J_0], J_pos))
    n = np.arange(0, N + 1)
    hamm = 0.54 - 0.46 * np.cos(2 * np.pi * n / N)
    hw = J * hamm
    hw = hw / np.sqrt(np.sum(np.abs(hw) ** 2))
    return hw

def generate_rayleigh_envelope(signal_length, fd):
    Fs = 10000  # Sampling frequency
    N = 512
    Ts = 1 / Fs
    h = Jakes_filter(fd, Ts, N)  # Jakes filter
    x = np.random.randn(signal_length + 1000)
    y = np.convolve(x, h, 'valid')
    return y


In [7]:
def apply_rayleigh_fading(signal, y):
    signal2 = copy.deepcopy(signal)
    for i in range(len(signal2)):
        signal2[i] *= y[i]
    return signal2, y[:len(signal2)]


In [9]:
def add_awgn_noise(signal, snr_db):
    snr = 10 ** (snr_db / 10)
    signal_power = np.mean(signal ** 2)
    noise_power = signal_power / snr
    noise = np.sqrt(noise_power) * np.random.normal(size=signal.shape)
    return signal + noise


In [11]:
def lstm_predict_h(y_values, window_length):
    model = Sequential()
    model.add(LSTM(50, input_shape=(window_length, 1)))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    
    X = []
    y = []
    for i in range(window_length, len(y_values)):
        X.append(y_values[i-window_length:i])
        y.append(y_values[i])
        
    X, y = np.array(X), np.array(y)
    X = X.reshape((X.shape[0], X.shape[1], 1))
    
    model.fit(X, y, epochs=5, batch_size=32, verbose=0)
    
    # Predict next h values
    predictions = []
    for i in range(window_length, len(y_values)):
        predictions.append(model.predict(X[i-window_length].reshape(1, window_length, 1), verbose=0)[0][0])
    
    return predictions

In [13]:
def run_lstm_prediction():
    signal_length = 51200
    packet_size = 256
    snr_db = 10 * np.log10(1000)
    doppler_frequencies = np.arange(10, 310, 10)
    window_lengths = np.arange(5, 205, 5)
    
    mse_results = []
    
    original_signal = generate_digital_signal(signal_length)

    for fd in doppler_frequencies:
        rayleigh_envelope = generate_rayleigh_envelope(signal_length, fd)
        faded_signal, actual_y = apply_rayleigh_fading(original_signal, rayleigh_envelope)
        noisy_signal = add_awgn_noise(faded_signal, snr_db)
        
        for window_length in window_lengths:
            predicted_h = lstm_predict_h(actual_y, window_length)
            predicted_h = np.concatenate((actual_y[:window_length], predicted_h))  # Aligning the predicted h
            
            # Applying estimated h (from LSTM) to the noisy signal
            estimated_signal = apply_channel_estimation(noisy_signal, predicted_h[:len(noisy_signal) // packet_size])
            estimated_signal2 = digitize(estimated_signal)
            
            mse = mean_squared_error(original_signal, estimated_signal2)
            mse_results.append((fd, window_length, mse))
    
    # Plotting results
    mse_results = np.array(mse_results)
    plt.figure(figsize=(14, 7))
    
    for fd in doppler_frequencies:
        plt.plot(mse_results[mse_results[:, 0] == fd][:, 1], mse_results[mse_results[:, 0] == fd][:, 2], label=f'Doppler Frequency = {fd} Hz')
    
    plt.title('MSE vs Window Length for Different Doppler Frequencies')
    plt.xlabel('Window Length')
    plt.ylabel('MSE')
    plt.legend()
    plt.grid(True)
    plt.show()


In [15]:
run_lstm_prediction()

C:\Users\vigne\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


NameError: name 'apply_channel_estimation' is not defined